<a href="https://colab.research.google.com/github/andrewcgaitskell/dmtoolnotes/blob/main/SeparateDropDown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%sh
pip install -q ipywidgets
pip install -q dash
pip install -q dash_core_components
pip install -q dash_bootstrap_components
pip install -q dash_table
pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=6882d830f36f6f86696ca7428849baa58e8be446236ea0098de64e700d1fe6d2
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [2]:
import dash
from dash import dcc
from dash import html
from dash import dash_table

import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from dash import Dash, Input, Output, callback, State

from jupyter_dash import JupyterDash

import dash_bootstrap_components as dbc
##COMPONENT_STYLE = "/assets/my_component.css"
# COMPONENT_STYLE = "/assets/table.css"
external_stylesheets=[dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__,##requests_pathname_prefix="/dash1/",routes_pathname_prefix='/dash1/',
                  external_stylesheets=external_stylesheets,
                  meta_tags=[{'name': 'viewport', 'content': 'width=device-width, initial-scale=1'}],
                 suppress_callback_exceptions=True)
# Create server variable with Flask server object for use with gunicorn
server = app.server

In [3]:
import pandas as pd

In [4]:
original_df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv')
original_df = original_df[['continent', 'country', 'pop', 'lifeExp']]  # prune columns for example
original_df

,continent,country,pop,lifeExp
0,Asia,Afghanistan,31889923.0,43.828
1,Europe,Albania,3600523.0,76.423
2,Africa,Algeria,33333216.0,72.301
3,Africa,Angola,12420476.0,42.731
4,Americas,Argentina,40301927.0,75.320
...,...,...,...,...
137,Asia,Vietnam,85262356.0,74.249
138,Asia,West Bank and Gaza,4018332.0,73.422
139,Asia,"Yemen, Rep.",22211743.0,62.698
140,Africa,Zambia,11746035.0,42.384


In [5]:
original_df['Mock Date'] = [
    datetime.datetime(2020, 1, 1, 0, 0, 0) + i * datetime.timedelta(hours=13)
    for i in range(len(original_df))]

In [6]:

def get_band(lifeExp):
    band_raw = 10 * int(lifeExp/10)
    return band_raw

In [7]:
original_df['band'] = original_df.apply(lambda x: get_band(x['lifeExp']), axis=1)
band_list = original_df['band'].unique().tolist()
band_list
band_df = pd.DataFrame(data=band_list, columns=['band'])
#continent_list
band_df

,band
0,40
1,70
2,80
3,60
4,50
5,30


In [8]:
continent_list = original_df['continent'].unique().tolist()
continent_df = pd.DataFrame(data=continent_list, columns=['continent'])
#continent_list
continent_df

,continent
0,Asia
1,Europe
2,Africa
3,Americas
4,Oceania


In [9]:
table = dash_table.DataTable(
    id='datatable1',
    columns=[
        {'name': 'Continent', 'id': 'continent', 'type': 'numeric'},
        {'name': 'Country', 'id': 'country', 'type': 'text'},
        {'name': 'Population', 'id': 'pop', 'type': 'numeric'},
        {'name': 'Life Expectancy', 'id': 'lifeExp', 'type': 'numeric'},
        {'name': 'Mock Dates', 'id': 'Mock Date', 'type': 'datetime'},
        {'name': 'band', 'id': 'band', 'type': 'numeric'}
    ],
    data=original_df.to_dict('records'),
    selected_rows=[],
    filter_action='native',
    page_size=10,
    style_table={
        'height': 400,
    },
    style_data={
        'width': '150px', 'minWidth': '150px', 'maxWidth': '150px',
        'overflow': 'hidden',
        'textOverflow': 'ellipsis',
    }
)


In [10]:
continent_table = dash_table.DataTable(
    id='continent_table',
    columns=[
        {'name': 'Continent', 'id': 'continent', 'type': 'text'},
    ],
    data=continent_df.to_dict('records'),
    filter_action='native',
    row_selectable='multi',
    selected_rows=[],
    style_table={
        'height': 150,
    },
    style_data={
        'width': '25px', 'minWidth': '25px', 'maxWidth': '25px',
        ##'overflow': 'hidden',
        ##'textOverflow': 'ellipsis',
    }
)


lifeexp_table = dash_table.DataTable(
    id='lifeexp_table',
    columns=[
        {'name': 'band', 'id': 'band', 'type': 'text'},
    ],
    data=band_df.to_dict('records'),
    filter_action='native',
    row_selectable='multi',
    selected_rows=[],
    style_table={
        'height': 150,
    },
    style_data={
        'width': '25px', 'minWidth': '25px', 'maxWidth': '25px',
        ##'overflow': 'hidden',
        ##'textOverflow': 'ellipsis',
    }
)

In [11]:
continent_dropdown_div = html.Div([
    dcc.Dropdown(continent_list, 'Europe', id='continent-dropdown')
])

lifeexp_dropdown_div = html.Div([
    dcc.Dropdown(band_list, '70', id='lifeexp-dropdown')
])

In [12]:
row0 = dbc.Row(
    [
        dbc.Alert(id='tbl_out'),
    ],
    className="g-0",
)


row1 = dbc.Row(
    [
        dbc.Col(html.Div("One of three columns",style={'background-color':'red'})),
        dbc.Col(html.Div("One of three columns",style={'background-color':'green'})),
        dbc.Col(html.Div("One of three columns",style={'background-color':'blue'})),
    ],
    className="g-0",
)

row2 = dbc.Row(
    [
        dbc.Col(continent_dropdown_div),
        dbc.Col(lifeexp_dropdown_div),
        dbc.Col(html.Div("One of three columns",style={'background-color':'blue','height':'100%'})),
    ],
    className="g-0",
)

row3 = dbc.Row(
    [
        dbc.Col(continent_table,width=2),
        dbc.Col(lifeexp_table,width=2),
        dbc.Col(html.Div("One of six columns",style={'background-color':'red','height':'100%'}), width=2),
        dbc.Col(html.Div("One of six columns",style={'background-color':'orange','height':'100%'}), width=2),
        dbc.Col(html.Div("One of six columns",style={'background-color':'yellow','height':'100%'}), width=2),
        dbc.Col(html.Div("One of six columns",style={'background-color':'green','height':'100%'}), width=2),
    ],
    className="g-0",
)

row4 = dbc.Row(
    [
        dbc.Col(table),
    ],
    className="g-0",
)


#app.layout =  html.Div([row1,row2,row3,row4])

In [13]:
#app.run_server(mode='inline', debug=True)

app.layout = html.Div([continent_dropdown_div,lifeexp_dropdown_div, continent_table, table])

@app.callback(
    output=Output('datatable1', "data"),
    inputs=[Input('continent-dropdown', 'value'), Input('lifeexp-dropdown', 'value')]
    )
def update_graph(continent,lifeband):
    if continent is None:
      dff1 = df;
    else:
      dff1 = df[df['continent']==continent]

    if lifeband is None:
      dff1 = dff1;
    else:
      dff1 = dff1[dff1['band']==lifeband]

    data1=dff1.to_dict("records")

    return data1


app.layout =  html.Div([row0,row1,row2,row3,row4])

@app.callback(
    ##Output('datatable1', 'data'),
    Output(component_id='tbl_out', component_property='children'),
    Input('continent_table', 'derived_virtual_row_ids'),
    Input('continent_table', 'active_cell'),
    ##Input('lifeexp_table', 'derived_virtual_row_ids'),
    ##Input('lifeexp_table', 'active_cell')
    )
def update_graphs(continent_selected_row_ids, continent_active_cell):##,lifeexp_selected_row_ids, lifeexp_active_cell):
       
    #dff1 = original_df.loc[continent_row_ids]
    ##selected_continents_df = continent_df.loc[continent_selected_row_ids]
    ##selectedcontinent_list = list(selected_continents_df['continent'])
    ##dff1 = original_df[original_df['continent'].isin(selectedcontinent_list)]

    #active_row_id = active_cell['row_id'] if active_cell else None

    ##data1=dff1.to_dict("records")

    return str(continent_selected_row_ids) if continent_selected_row_ids else "Select a continent"

 # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncrasy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.

    if continent_selected_row_ids is None:
        dff1 = original_df
        # pandas Series works enough like a list for this to be OK
        row_ids = original_df['id']
    else:

In [14]:
selected_rows_1 = [0, 1, 2]
selected_continents_df = continent_df.loc[selected_rows_1]
selectedcontinent_list = list(selected_continents_df['continent'])
selectedcontinent_list
justcont_df = original_df[original_df['continent'].isin(selectedcontinent_list)]
justcont_df


,continent,country,pop,lifeExp,Mock Date,band
0,Asia,Afghanistan,31889923.0,43.828,2020-01-01 00:00:00,40
1,Europe,Albania,3600523.0,76.423,2020-01-01 13:00:00,70
2,Africa,Algeria,33333216.0,72.301,2020-01-02 02:00:00,70
3,Africa,Angola,12420476.0,42.731,2020-01-02 15:00:00,40
6,Europe,Austria,8199783.0,79.829,2020-01-04 06:00:00,70
...,...,...,...,...,...,...
137,Asia,Vietnam,85262356.0,74.249,2020-03-15 05:00:00,70
138,Asia,West Bank and Gaza,4018332.0,73.422,2020-03-15 18:00:00,70
139,Asia,"Yemen, Rep.",22211743.0,62.698,2020-03-16 07:00:00,60
140,Africa,Zambia,11746035.0,42.384,2020-03-16 20:00:00,40


In [15]:
app.layout = html.Div([row0,row1,row2,row3,row4])

@callback(
    Output('datatable1', 'data'),
    Output(component_id='tbl_out', component_property='children'),
    Input('continent_table', 'active_cell'),
    Input('continent_table', 'derived_virtual_selected_rows'),
    Input('lifeexp_table', 'active_cell'),
    Input('lifeexp_table', 'derived_virtual_selected_rows'),
    )
def update_graphs(active_cell_continent,derived_virtual_selected_rows_continent,active_cell_lifeexp,derived_virtual_selected_rows_lifeexp):
    
    selected_continents_df = continent_df.loc[derived_virtual_selected_rows_continent]
    selectedcontinent_list = list(selected_continents_df['continent'])

    selected_band_df = band_df.loc[derived_virtual_selected_rows_lifeexp]
    selected_band_list = list(selected_band_df['band'])
    
    if not selectedcontinent_list:
        dff1 = original_df
    else:
        dff1 = original_df[original_df['continent'].isin(selectedcontinent_list)].copy()

    if not selected_band_list:
        dff2 = dff1.copy()
    else:
        dff2 = dff1[dff1['band'].isin(selected_band_list)].copy()

    #active_row_id = active_cell['row_id'] if active_cell else None

    data1=dff2.to_dict("records")
    list_output = str(selectedcontinent_list) if selectedcontinent_list else "Click the table"
    return data1 , list_output

In [16]:
app.run_server(mode='inline', debug=True)

<IPython.core.display.Javascript object>